In [1]:
import os
print(os.getcwd())
import sys
sys.path.append('/local/scratch/kchan76/sparse-coarse-operator/')
from libs.pde import *
from libs.rotated_laplacian import RotatedLaplacian
from libs.models import BasicNet, MultiHeadedAttention
import torch

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('device:', device)

/local/scratch/kchan76/sparse-coarse-operator/tests/laplacian/fixed-xi


/local/scratch/kchan76/sparse-stencil-learning/src/pyamg2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda:1


# 3-Level Attention 

In [48]:
list_eps = [1/200]
low_pi = 3
high_pi = 4
theta_low = np.pi/12*low_pi
theta_high = np.pi/12*high_pi
npts = 5
list_theta = np.linspace(theta_low, theta_high, npts)

In [49]:
train_grid_size = 31
k2 = 20
k3 = 5
same_vecs = False
PDE = RotatedLaplacian(train_grid_size,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

In [50]:
# LEVEL 2
seed = 4
print('seed:', seed)
set_global_seed(seed)
h = 8
dhid = 256
epochs = 1250
adam_decay_rate = 0 # adam weight decay
lr = 1e-4 # learning rate
lr_decay_rate = 1 # learning rate decay
lr_decay_step = 700 # learning rate decay step size
init = 'xavier uniform' # "<name> <distribution>" name={'xavier', 'kaiming'} distribution={'uniform', 'normal'}

ncopy = 1
single_model = False # KEEP IT FALSE!! if true, only model_prob will be learned
softmax_topk = False
dropout = 0.8
enforce_stencil_symmetry = 'x'
model_prob_l2 = MultiHeadedAttention(h ,dhid, ncopy=ncopy, init=init, dropout=dropout)
model_value_l2 = MultiHeadedAttention(h, dhid, ncopy=ncopy, init=init, dropout=dropout)


model_prob_l2,model_value_l2 = PDE.train(PDE.A2_train, 
                                         PDE.s2_train,
                                         PDE.eig_vec2_train,
                                         model_prob_l2,
                                         model_value_l2,
                                         epochs,
                                         adam_decay_rate,
                                         lr,
                                         lr_decay_rate,
                                         lr_decay_step,
                                         device,
                                         single_model=single_model,
                                         softmax_on = softmax_topk,
                                         enforce_stencil_symmetry = enforce_stencil_symmetry,
                                         verbose = True)




seed: 4
 epoch:  0  loss:  tensor(16.6179, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0390, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0312, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0411, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0294, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1249  loss:  tensor(0.0350, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


In [24]:
# test level-2 model
test_seed = 64
# while True:
# test_seed = test_seed + 1
set_global_seed(test_seed)
# sizes = [63,127,255,511]
sizes = [511]
print('test seed:', test_seed)
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
test_grid_size = 63
max_levels = 2
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
random_test = True
accel = 'gmres'
dropout_on = False
enforce_stencil_symmetry = 'x'


for test_grid_size in sizes:
    print('size:', test_grid_size)
    num_iter_standard, num_iter_learning, thetaList, xs, xl = PDE.test_model(num_test,
                                                                    models,
                                                                    test_grid_size,
                                                                    max_levels,
                                                                    epsilon,
                                                                    theta,
                                                                    device,
                                                                    random_test=random_test,
                                                                    single_model=single_model,
                                                                    softmax_on = softmax_topk,
                                                                    enforce_stencil_symmetry = enforce_stencil_symmetry,
                                                                    dropout_on=dropout_on,
                                                                    top_accel = accel,
                                                                    verbose = True)
#     criterion = np.greater(num_iter_standard, num_iter_learning)
#     if not np.any(~criterion):
#     if np.mean(num_iter_learning)-np.mean(num_iter_standard) < 1:
#         seedlist.append(test_seed)
#         print(seedlist)
#     break


test seed: 64
size: 511
standard stencil iter:    11.3   standard stencil time:   1.9951786041259765
learned stencil iter:    97.7   learned stencil time:   12.905337762832641


In [101]:
print(num_iter_standard)
print(num_iter_learning)
# print(thetaList)

[20, 22, 22, 21, 21, 20, 20, 20, 22, 21]
[28, 31, 31, 29, 30, 30, 29, 31, 30, 29]


In [27]:
np.mean(num_iter_learning)

285.9

## Level 3

In [51]:
# LEVEL 3
l3_seed = 62
print('seed:', l3_seed)
set_global_seed(l3_seed)
h3 = 8
dhid3 = 256
epochs3 = 1000
adam_decay_rate3 = 0 # adam weight decay
lr3 = 1e-4 # learning rate
lr_decay_rate3 = 1 # learning rate decay
lr_decay_step3 = 500 # learning rate decay step size
init3 = 'xavier uniform' 

ncopy3 = 1
single_model3 = False
softmax_topk3 = True
dropout3 = 0.8
enforce_stencil_symmetry = 'x'

model_prob_l3 = MultiHeadedAttention(h3 ,dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)
model_value_l3 = MultiHeadedAttention(h3, dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)

model_prob_l3,model_value_l3 = PDE.train(PDE.A3_train, 
                                         PDE.s3_train,
                                         PDE.eig_vec3_train,
                                         model_prob_l3,
                                         model_value_l3,
                                         epochs3,
                                         adam_decay_rate3,
                                         lr3,
                                         lr_decay_rate3,
                                         lr_decay_step3,
                                         device,
                                         single_model= single_model3,
                                         softmax_on = softmax_topk3,
                                         enforce_stencil_symmetry = enforce_stencil_symmetry,
                                         verbose = True)

seed: 62
 epoch:  0  loss:  tensor(0.9383, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0012, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0033, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0020, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  999  loss:  tensor(0.0013, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


In [52]:
# import os
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
# if not os.path.isdir(MODEL_DIR):
#     os.makedirs(MODEL_DIR)
# torch.save(model_prob_l2, MODEL_DIR+"level2_prob.pt")
# torch.save(model_value_l2, MODEL_DIR+"level2_value.pt")
# torch.save(model_prob_l3, MODEL_DIR+"level3_prob.pt")
# torch.save(model_value_l3, MODEL_DIR+"level3_value.pt")

In [53]:
# MODEL_DIR = f"./models/laplacian/fixed-xi/xi-{int(1/list_eps[0])}/theta-{low_pi}pi\\12-{high_pi}pi\\12/"
# model_prob_l2=torch.load(MODEL_DIR+"level2_prob.pt")
# model_value_l2=torch.load(MODEL_DIR+"level2_value.pt")
# model_prob_l3=torch.load(MODEL_DIR+"level3_prob.pt")
# model_value_l3 = torch.load(MODEL_DIR+"level3_value.pt")

In [54]:
test_seed = 64
# while True:
#     test_seed = test_seed + 1
set_global_seed(test_seed)
print('test seed:', test_seed)
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
models['level3'] = model_prob_l3, model_value_l3
test_grid_size = 95
random_test = True
# sizes = [63,95,127,191,255,383,511]
sizes = [511]
accel = "gmres"
dropout_on = False
enforce_stencil_symmetry = 'x'

for test_grid_size in sizes:
    print('size: ',test_grid_size)
    set_global_seed(test_seed)
    max_levels = 3
    epsilon = tuple(list_eps)
    theta = (theta_low, theta_high)

    num_iter_standard, num_iter_learning, ThetaList,_, _ = PDE.test_model(num_test,
                                                                models,
                                                                test_grid_size,
                                                                max_levels,
                                                                epsilon,
                                                                theta,
                                                                device,
                                                                top_accel = accel,
                                                                random_test = random_test,
                                                                single_model=single_model,
                                                                softmax_on = softmax_topk,
                                                                enforce_stencil_symmetry = enforce_stencil_symmetry,
                                                                dropout_on=dropout_on,
                                                                verbose = True)

test seed: 64
size:  511
standard stencil iter:    14.9   standard stencil time:   1.0320589542388916
learned stencil iter:    35.6   learned stencil time:   2.929522490501404


In [40]:
print(num_iter_standard)
print(num_iter_learning)

[16, 15, 16, 16, 16, 16, 16, 16, 16, 16]
[28, 51, 29, 39, 39, 43, 35, 28, 34, 30]


# MLP

In [72]:
# define problem
list_eps = [0.01]
theta_low = np.pi/6
theta_high = np.pi/4
list_theta = np.linspace(theta_low, theta_high, 10)

In [74]:
# seed += 1
print('seed:',seed)
set_global_seed(seed)
epochs = 2000
adam_decay_rate = 0 # adam weight decay
lr = 1e-3 # learning rate
lr_decay_rate = 0.5 # learning rate decay
lr_decay_step = 500 # learning rate decay step size
model_prob_l2 = BasicNet([64])
model_value_l2 = BasicNet([64])
k2 = 10
k3 = 15
same_vecs = False
single_model = False # if true, only model_prob will be learned. 
softmax_on = True # if true, turn on the softmax in the function top_k during sparsfication
problem = RotatedLaplacian(31,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

model_prob_l2,model_value_l2 = problem.train(problem.A2_train, 
                                             problem.s2_train,
                                             problem.eig_vec2_train,
                                             model_prob_l2,
                                             model_value_l2,
                                             epochs,
                                             adam_decay_rate,
                                             lr,
                                             lr_decay_rate,
                                             lr_decay_step,
                                             device,
                                             single_model=single_model,
                                             softmax_on = softmax_on,
                                             verbose = True)

seed: 36
 epoch:  0  loss:  tensor(6.7295, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0111, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0104, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0101, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0098, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0096, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1500  loss:  tensor(0.0095, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1750  loss:  tensor(0.0094, device='cuda:0', grad_fn=<AddBackward0>)
 epoch:  1999  loss:  tensor(0.0093, device='cuda:0', grad_fn=<AddBackward0>)
